In [182]:
import os
import pandas as pd
import numpy as np
import pickle
import collections
from Graham.Aggregation import OuterAgg as Aggregate
pd.set_option('mode.chained_assignment', None)
pd.set_option('display.width', 1500)

In [223]:
symbols = np.array(os.listdir('/Users/duncangh/PycharmProjects/FSA/data/financials_data/')[1:])

In [195]:
class Standardize:
    def __init__(self, sym, rf=1.5):
        self.symbol = sym
        self.statements = Aggregate(sym).statements
        self.risk_free = rf

        self.balance = self.statements[0] / 10 ** 6
        self.income = self.statements[1] / 10 ** 6
        self.cash = self.statements[2] / 10 ** 6

        self.fcfe = self.free_cash_flow_equity()


    def change_nwc(self):
        """
        Calculate Change in Net Working Capital. There are two main ways to do this:

            1. From The Statement of Cash Flows
            2. From The Balance Sheet
        """
        if 'AssetsCurrent' in self.balance.index:
            net = self.balance.loc['AssetsCurrent'] - self.balance.loc['LiabilitiesCurrent']
        else:
            try:
                net = (self.balance.loc['Assets'] - self.balance.loc['PropertyPlantAndEquipmentNet']) - (self.balance.loc['Liabilities'] - self.balance.loc['DebtAndCapitalLeaseObligations'])
            except:
                print("No Change NWC For this stock")

        self.chg_wc = net - net.shift(1)

        return self.chg_wc.fillna(0)

    def CapEx(self):
        self.capex = self.cash.loc[self.cash.index[self.cash.index.str.contains('PropertyPlantAndE')]]

        if self.capex.empty:
            return 0
        else:
            self.capex = self.capex.loc[self.capex.index.values[0]]
            return self.capex.fillna(0)

    def debt_cash(self):
        issued = self.cash.loc[
            self.cash.index[(self.cash.index.str.contains('Debt')) & (self.cash.index.str.contains('Iss'))]]
        if not issued.empty:
            issued = issued.loc[issued.index[0]].fillna(0)

        payments = self.cash.loc[
            self.cash.index[(self.cash.index.str.contains('Debt')) & (self.cash.index.str.contains('Repay'))]]
        if not payments.empty:
            payments = payments.loc[payments.index[0]].fillna(0)

        if payments.empty:
            return issued

        elif issued.empty:
            return payments

        else:
            return (issued - payments).fillna(0)

    def DeprAmort(self):
        DA = self.cash.loc[
            self.cash.index[(self.cash.index.str.contains('Depre')) | (self.cash.index.str.contains('Amort'))]]
        if not DA.empty:
            return DA.max()
        else:
            return 0

    def interest_expense(self):
        # Handle all known cases of interest expense
        int_exp = self.income.loc[self.income.index[(self.income.index.str.contains('Interest'))
                                                    & (~self.income.index.str.contains('Minor'))
                                                    & (~self.income.index.str.contains('controll'))
                                                    & (~self.income.index.str.contains('Before'))
                                                    & (~self.income.index.str.contains('Gain'))
                                                    & (~self.income.index.str.contains('TotalRevenues'))
                                                    & (~self.income.index.str.contains('Investment'))
                                                    & (~self.income.index.str.contains('Nonopera'))
                                                    & (~self.income.index.str.contains('Income'))].values[0]]

        if not int_exp.empty:
            return int_exp.fillna(0)
        else:
            return 0

    def ebit(self):
        """
        Calculate the EBIT

        """
        try:
            EBIT = self.income.loc['OperatingIncomeLoss'].fillna(0)
        except:
            EBIT = self.income.loc[self.income.index[self.income.index.str.contains('OperatingIncome')]].fillna(0)
            if EBIT.empty:
                try:
                    EBIT = self.income.loc['NetIncomeLoss'] + self.income.loc['IncomeTaxExpenseBenefit'] + self.interest_expense()
                    EBIT = EBIT.fillna(0)
                except:
                    pass

        return EBIT

    def free_cash_flow_equity(self):
        """
        FCFe = NI + D/A - ∆ nwc - CapEx  + Net Debt
        """

        try:
            NI = self.income.loc['NetIncomeLoss'].fillna(0)
        except:
            try:
                NI = self.cash.loc['NetIncomeLoss'].fillna(0)
            except:
                NI = self.income.loc[self.income.index[self.income.index.str.contains('NetIncome')].values[0]].fillna(0)

        DA = self.DeprAmort()
        nwc_chg = self.change_nwc()
        capex = self.CapEx()

        net_debt = self.debt_cash()
        if net_debt.empty:
            net_debt = 0

        NIDA = ((NI + DA).fillna(0) + net_debt).fillna(0)
        nwcap = (nwc_chg + capex).fillna(0)
        return (NIDA - nwcap)

    def free_cash_flow_firm(self):
        """
        FCFf = EBIT + Interest Expense + D/A - ∆ nwc - CapEx
        """

        EBIT = self.ebit()

        DA = self.DeprAmort()
        nwc_chg = self.change_nwc()
        capex = self.CapEx()

        int_exp = self.interest_expense()

        EBITDA = ((EBIT + DA).fillna(0) + int_exp).fillna(0)
        nwcap = (nwc_chg + capex).fillna(0)
        return (EBITDA - nwcap)


    def book_value_debt(self):
        """
        Book Value of Debt = (Assets - Equity) - Non-Interest Bearing Obligations

        Use this method to perform the above calculation. Should be robust to natural
        Variety in the names of common Non-Interest Bearing Obligations.

        """

        total_liabilities = self.balance.loc['LiabilitiesAndStockholdersEquity'].fillna(0) - self.balance.loc['StockholdersEquity'].fillna(0)
        if 'AccountsPayableCurrent' in self.balance.index:
            return total_liabilities - self.balance.loc['AccountsPayableCurrent'].fillna(0)

        else:
            return total_liabilities.fillna(0)

    def book_value_equity(self):
        """
        Book Value of Equity = Value of Stockholders Equity

        """
        return self.balance.loc['StockholdersEquity'].fillna(0)

    def cost_debt(self, debt):
        """
        Calculate the cost of debt Using
            1. amount of debt
            2. interest expense
            3. Tax Rate if possible

        """
        interest = self.interest_expense()
        earnings_before_taxes = (self.ebit() - interest).fillna(0)
        tax_rate = self.income.loc['IncomeTaxExpenseBenefit'].fillna(0) / earnings_before_taxes

        interest_rate = (interest / debt) * 100
        
        if 0 < tax_rate.mean() < 1:
            return (interest_rate * (1 - tax_rate)).fillna(0)
        else:
            return interest_rate.fillna(0)

    def cost_equity(self):
        """
        Rrf + ß(Rm - Rrf)

        MRP is assumed to be 4.5%
        """
#         beta = pd.read_pickle('/Users/duncangh/PycharmProjects/FSA/Graham/Beta.Pkl')[self.symbol].abs()
        beta = pd.read_pickle('/Users/duncangh/PycharmProjects/FSA/Graham/Beta.Pkl')['SPY'].abs()
        beta.index = beta.index.astype(str)
        return self.risk_free + beta*(4.5 - self.risk_free)




    def weighted_average_cost_of_capital(self):
        """
        Total Capital = BV(equity) + BV(debt)
        Book Value of Equity = Assets - Liabilities
        Book Value of Debt = (Assets - Equity) - Non-Interest Bearing Obligations

        Equity Cost = Rrf + ß(Rm - Rrf)
        Cost of Debt = (Interest Expense / Book Value of Debt) * (1 - MarginalTaxRate)

        Weight of Equity = BV Equity / Total Capital
        Weight of Debt = BV of Debt / Total Capital

        WACC = (WeightofEquity * CostofEquity) + (WeightofDebt * CostofDebt)
        WACC = rD(1 - Tc) * (D / V) + rE * (E / V)

        :return: WACC
        """
        book_value_debt = self.book_value_debt()
        book_value_equity = self.book_value_equity()
        
        total_capital = book_value_debt + book_value_equity

        weight_debt = book_value_debt / total_capital
        weight_equity = book_value_equity / total_capital


        cost_debt = self.cost_debt(book_value_debt)
        cost_equity = self.cost_equity()
        
        wacc = (weight_debt * cost_debt) + (weight_equity * cost_equity)
        
        return weight_debt, weight_equity, cost_debt, cost_equity, wacc
    
    def make_model(self):
        wd, we, cd, ce, wacc = self.weighted_average_cost_of_capital()
        fcfe = stock.free_cash_flow_equity()
        fcff = stock.free_cash_flow_firm()
        model = pd.DataFrame({'FCFF' : fcff, 'FCFE' : fcfe})
        model = model.assign(Cost_Equity=ce, Cost_Debt=cd, Weight_Equity=we, Weight_Debt=wd, WACC=wacc)
        model = model.fillna(method='bfill')
        return model

In [284]:
stock = Standardize('AFL') 
stock.make_model()
# Currently using a Beta of one, so cost of equity is the same for all stocks. 

# ['ADS', 'AFL', 'AGN', 'AMZN', 'AON', 'AVY', 'AXP', 'BA', 'BBY', 'BEN', 'BHI', 'BIIB', 'BLL', 'CHRW', 'CL', 'CRM', 'CXO', 'D',       'DLPH', 'EBAY', 'ED', 'ESS', 'ETFC', 'ETN', 'EXPD', 'FLS', 'FTV',
#        'JEC', 'JNJ', 'KHC', 'LH', 'MMM', 'MSFT', 'NFLX', 'NVDA', 'ORCL', 'T', 'TSLA']

,FCFE,FCFF,Cost_Debt,Cost_Equity,WACC,Weight_Debt,Weight_Equity
2007,2641.0,3883.0,0.000000,4.5,0.611877,0.888624,0.111376
2008,2641.0,3883.0,0.000000,4.5,0.611877,0.888624,0.111376
2009,2641.0,3883.0,0.000000,4.5,0.611877,0.888624,0.111376
2010,2641.0,3883.0,inf,4.5,0.611877,0.888624,0.111376
2011,46.0,1308.0,0.124559,4.5,0.611877,0.888624,0.111376
2012,-121.0,672.0,0.151046,4.5,0.681106,0.878118,0.121882
2013,4588.0,6132.0,0.180087,4.5,0.700726,0.879479,0.120521
2014,-798.0,961.0,0.205382,4.5,0.863270,0.846811,0.153189
2015,3167.0,5348.0,0.188441,4.5,0.833848,0.850308,0.149692


In [196]:
failed = []
for symbol in symbols:
    try:
        stock = Standardize(symbol)
        print(stock.make_model())
    except:
        failed.append(symbol)
failed

          FCFE     FCFF  Cost_Debt  Cost_Equity      WACC  Weight_Debt  Weight_Equity
2008 -2945.472  677.977   0.000000          4.5  2.139854     0.946742       0.053258
2009 -2945.472  677.977   2.007086          4.5  2.139854     0.946742       0.053258
2010 -3354.332 -947.078   2.464807          4.5  2.470574     0.997166       0.002834
2011 -2999.667  779.969   2.115869          4.5  2.163378     0.980073       0.019927
2012 -1224.385   85.130   1.601047          4.5  1.731051     0.955155       0.044845
2013 -2352.158 -168.447   1.568736          4.5  1.761188     0.934345       0.065655
2014 -2581.616   51.126   0.921633          4.5  1.354538     0.879022       0.120978
2015 -3445.675 -251.381   1.074457          4.5  1.387725     0.908549       0.091451
        FCFE    FCFF  Cost_Debt  Cost_Equity      WACC  Weight_Debt  Weight_Equity
2007  2641.0  3883.0   0.000000          4.5  0.611877     0.888624       0.111376
2008  2641.0  3883.0   0.000000          4.5  0.611877     0

['AGN',
 'AVY',
 'AXP',
 'BA',
 'BEN',
 'BIIB',
 'CHRW',
 'CXO',
 'ESS',
 'ETFC',
 'ETN',
 'FTV',
 'GE',
 'GPC',
 'HAL',
 'HCP',
 'HPE',
 'HST',
 'JNJ',
 'KHC',
 'MSFT',
 'T']

In [215]:
len(failed)

22

In [250]:
stock = failed[21]
print(stock)
Standardize(stock).make_model()


T
No Change NWC For this stock


UnboundLocalError: local variable 'net' referenced before assignment

In [248]:
biib = Aggregate('KHC')
biib.income_statement / 10**6
# os.listdir('/Users/duncangh/PycharmProjects/FSA/data/financials_data/GE/10-K/xml/2011-12-31')

,2013,2014,2016
Fiel,,,


In [257]:
l = []
nwc_stocks = ['HST', 'T', 'AXP', 'BA', 'BEN', 'ESS', 'ETFC', 'HCP', 'HPE']
for sym in nwc_stocks:
    l.extend(Aggregate(sym).balance_sheet.index.values)
df = pd.DataFrame(l)

In [261]:
symbols

array(['ADS', 'AFL', 'AGN', 'AMZN', 'AON', 'AVY', 'AXP', 'BA', 'BBY',
       'BEN', 'BHI', 'BIIB', 'BLL', 'CHRW', 'CL', 'CRM', 'CXO', 'D',
       'DLPH', 'EBAY', 'ED', 'ESS', 'ETFC', 'ETN', 'EXPD', 'FLS', 'FTV',
       'GE', 'GPC', 'GWW', 'HAL', 'HCP', 'HPE', 'HST', 'HUM', 'INTU',
       'JEC', 'JNJ', 'KHC', 'LH', 'MMM', 'MSFT', 'NFLX', 'NVDA', 'ORCL',
       'T', 'TSLA'], 
      dtype='<U4')

In [267]:
stock = Standardize('ADS')
stock.change_nwc() 

2009       0.000
2010    2037.854
2011     484.449
2012    1347.570
2013    1788.978
2014    1620.949
2015    2335.565
dtype: float64

In [274]:
stock.balance.loc['Assets'] - stock.balance.loc['Liabilities']

2009     272.776
2010      23.094
2011     175.966
2012     528.487
2013     855.761
2014    2631.946
2015    2177.407
dtype: float64

2009     4952.891
2010     8249.058
2011     8804.283
2012    11471.652
2013    12388.496
2014    17632.031
2015    20244.423
Name: Liabilities, dtype: float64

In [45]:
stock.cash

,2008,2009,2010,2011,2012,2013,2014,2015,2016
Fiel,,,,,,,,,
AmortizationOfDeferredSalesCommissions,42.195,58.732,63.891,80.159,107.195,154.818,194.553,257.642,319.074
CashAndCashEquivalentsPeriodIncreaseDecrease,192.487,204.739,527.472,-587.014,182.992,139.961,34.390,126.482,250.246
DepreciationAndAmortization,24.219,35.971,53.177,75.746,157.286,216.795,369.423,448.296,525.750
EffectOfExchangeRateOnCashAndCashEquivalents,-1.792,-0.732,-1.976,2.385,5.325,7.437,-7.758,-38.391,-7.109
ExcessTaxBenefitFromShareBasedCompensationFinancingActivities,31.978,54.597,51.539,35.991,6.018,14.933,8.144,7.730,59.496
ExcessTaxBenefitFromShareBasedCompensationOperatingActivities,31.978,54.597,51.539,35.991,6.018,14.933,8.144,7.730,59.496
IncreaseDecreaseInAccountsPayable,-1.392,8.512,-1.588,1.246,12.644,0.000,0.000,0.000,0.000
IncreaseDecreaseInAccountsReceivable,91.368,44.798,54.522,102.507,244.947,183.242,424.702,544.610,582.425
IncreaseDecreaseInAccruedLiabilities,71.779,55.440,64.498,132.004,67.692,0.000,0.000,0.000,0.000


In [147]:
beta = pd.read_pickle('/Users/duncangh/PycharmProjects/FSA/Graham/Beta.Pkl')
beta.columns

Index(['A', 'AAL', 'AAP', 'AAPL', 'ABBV', 'ABC', 'ABT', 'ACN', 'ADBE', 'ADI', 'ADM', 'ADS', 'AEE', 'AEP', 'AES', 'AET', 'AFL', 'AGN', 'AIG', 'AIV', 'AJG', 'AKAM', 'ALB', 'ALK', 'ALL', 'ALLE', 'ALXN', 'AMAT', 'AME', 'AMG', 'AMGN', 'AMP', 'AMT', 'AMZN', 'ANTM', 'AON', 'APA', 'APC', 'APD', 'APH', 'ATVI', 'AWK', 'AXP', 'AYI', 'GOOG', 'GOOGL', 'LNT', 'MMM', 'MO'], dtype='object', name='Symbol')

In [12]:
(nvda.income.loc['Revenues'].pct_change() * 100).mean()
# (nvda.income.loc['NetIncomeLoss'].pct_change() * 100)
nvda.income

,2008,2009,2010,2011,2012,2013,2014,2015,2016
Fiel,,,,,,,,,
CostOfRevenue,2228.580,2250.590,2149.522,2134.219,1941.413,2053.816,1862.399,2082.030,2199.0
GrossProfit,1869.280,1174.269,1176.923,1409.090,2056.517,2226.343,2267.763,2599.477,2811.0
IncomeTaxExpenseBenefit,103.696,-12.913,-14.307,18.023,82.306,99.503,70.264,124.249,129.0
InterestExpense,0.054,0.406,3.320,3.127,3.089,3.294,10.443,46.133,47.0
InvestmentIncomeInterest,64.289,42.859,23.115,19.057,19.149,19.908,17.119,28.090,39.0
NetIncomeLoss,797.645,-30.041,-67.987,253.146,581.090,562.536,439.990,630.587,614.0
OperatingExpenses,1032.934,1244.969,1275.868,1153.343,1408.218,1578.104,1771.536,1840.488,2064.0
OperatingIncomeLoss,836.346,-70.700,-98.945,255.747,648.299,648.239,496.227,758.989,747.0
OtherNonoperatingIncomeExpense,0.760,-14.707,-3.144,-0.508,-0.963,-2.814,7.351,13.890,4.0


In [22]:
((nvda.income.pct_change(axis=1) * 100).dropna(axis=1)).applymap(lambda x: '%.2f' % x)

,2009,2010,2011,2012,2013,2014,2015,2016
Fiel,,,,,,,,
CostOfRevenue,0.99,-4.49,-0.71,-9.03,5.79,-9.32,11.79,5.62
GrossProfit,-37.18,0.23,19.73,45.95,8.26,1.86,14.63,8.14
IncomeTaxExpenseBenefit,-112.45,10.80,-225.97,356.67,20.89,-29.39,76.83,3.82
InterestExpense,651.85,717.73,-5.81,-1.22,6.64,217.03,341.76,1.88
InvestmentIncomeInterest,-33.33,-46.07,-17.56,0.48,3.96,-14.01,64.09,38.84
NetIncomeLoss,-103.77,126.31,-472.34,129.55,-3.19,-21.78,43.32,-2.63
OperatingExpenses,20.53,2.48,-9.60,22.10,12.06,12.26,3.89,12.14
OperatingIncomeLoss,-108.45,39.95,-358.47,153.49,-0.01,-23.45,52.95,-1.58
OtherNonoperatingIncomeExpense,-2035.13,-78.62,-83.84,89.57,192.21,-361.23,88.95,-71.20


In [28]:
income = nvda.income.loc[['Revenues', 'CostOfRevenue', 'GrossProfit', 'ResearchAndDevelopmentExpense', 'SellingGeneralAndAdministrativeExpense', 
                'OperatingExpenses', 'OperatingIncomeLoss', 'InterestExpense', 'IncomeTaxExpenseBenefit', 
                'InvestmentIncomeInterest', 'OtherNonoperatingIncomeExpense', 'NetIncomeLoss']]
income

,2008,2009,2010,2011,2012,2013,2014,2015,2016
Fiel,,,,,,,,,
Revenues,4097.860,3424.859,3326.445,3543.309,3997.930,4280.159,4130.162,4681.507,5010.0
CostOfRevenue,2228.580,2250.590,2149.522,2134.219,1941.413,2053.816,1862.399,2082.030,2199.0
GrossProfit,1869.280,1174.269,1176.923,1409.090,2056.517,2226.343,2267.763,2599.477,2811.0
ResearchAndDevelopmentExpense,691.637,855.879,908.851,848.830,1002.605,1147.282,1335.834,1359.725,1331.0
SellingGeneralAndAdministrativeExpense,341.297,362.222,367.017,361.513,405.613,430.822,435.702,480.763,602.0
OperatingExpenses,1032.934,1244.969,1275.868,1153.343,1408.218,1578.104,1771.536,1840.488,2064.0
OperatingIncomeLoss,836.346,-70.700,-98.945,255.747,648.299,648.239,496.227,758.989,747.0
InterestExpense,0.054,0.406,3.320,3.127,3.089,3.294,10.443,46.133,47.0
IncomeTaxExpenseBenefit,103.696,-12.913,-14.307,18.023,82.306,99.503,70.264,124.249,129.0


In [29]:
((income.pct_change(axis=1) * 100).dropna(axis=1)).applymap(lambda x: '%.2f' % x)

,2009,2010,2011,2012,2013,2014,2015,2016
Fiel,,,,,,,,
Revenues,-16.42,-2.87,6.52,12.83,7.06,-3.50,13.35,7.02
CostOfRevenue,0.99,-4.49,-0.71,-9.03,5.79,-9.32,11.79,5.62
GrossProfit,-37.18,0.23,19.73,45.95,8.26,1.86,14.63,8.14
ResearchAndDevelopmentExpense,23.75,6.19,-6.60,18.12,14.43,16.43,1.79,-2.11
SellingGeneralAndAdministrativeExpense,6.13,1.32,-1.50,12.20,6.22,1.13,10.34,25.22
OperatingExpenses,20.53,2.48,-9.60,22.10,12.06,12.26,3.89,12.14
OperatingIncomeLoss,-108.45,39.95,-358.47,153.49,-0.01,-23.45,52.95,-1.58
InterestExpense,651.85,717.73,-5.81,-1.22,6.64,217.03,341.76,1.88
IncomeTaxExpenseBenefit,-112.45,10.80,-225.97,356.67,20.89,-29.39,76.83,3.82


In [33]:
nvda.income.iloc[:, -1].sort_values(ascending=False)

Fiel
Revenues                                  5010.0
GrossProfit                               2811.0
CostOfRevenue                             2199.0
OperatingExpenses                         2064.0
ResearchAndDevelopmentExpense             1331.0
OperatingIncomeLoss                        747.0
NetIncomeLoss                              614.0
SellingGeneralAndAdministrativeExpense     602.0
IncomeTaxExpenseBenefit                    129.0
InterestExpense                             47.0
InvestmentIncomeInterest                    39.0
OtherNonoperatingIncomeExpense               4.0
Name: 2016, dtype: float64